In [1]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array, Process, current_process
import time
import os
import numpy as np
import random

In [32]:


def read_4bytes(filename):
    numbers = []
    with open(filename, 'br') as f:
        data = f.read(4)
        while data:
            number = int.from_bytes(data, "big", signed=True)
            numbers.append(number)
            data = f.read(4)
    return numbers



In [33]:
def count_frequency(x): 
    freq = {} 
    for item in x: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [34]:
def random_choices(x, tries):
    sequence = []
    while len(sequence) < tries:
        sequence.append(random.choice(x))
    return sequence

In [35]:
def make_subsample(num_items, x):
    resulting_sequence = []
    weights = []
    while num_items:
        number_of_tries = 1000000
        sequence = random_choices(x, number_of_tries)
        freq = count_frequency(sequence)
        highest = max(freq.values())
        for key, value in freq.items():
            if value == highest:
                resulting_sequence.append(key)
                weights.append(value / number_of_tries)
                x.remove(key)
                break ;
        random.shuffle(x)
        num_items -= 1
    return resulting_sequence


In [36]:
def check_the_pid(parameters):
    filename, N = parameters
    start_time = time.time()
    x  = read_4bytes(filename)
    time_for_reading = start_time - time.time()
    subsample = make_subsample(N, x)
    time_for_sampling = time_for_reading - time.time()
    pid = int(os.getpid())
    proc_file = str(current_process().name)+"-"+ str(pid)+ ".txt"
    f = open(proc_file, "a+")
    f.write("Process ID : %d\nParent ID : %d\n" % (pid, os.getppid()))
    res = []
    for i in subsample:
        f.write("%d\n" % (i))
        if i % (3 * pid) == 0:
            res.append(i)
    for i in res: f.write("%s\nФ" % (i))
    f.write("Execution time: %d\nTime spent for reading file: %d\nTime spent fot sampling: %d\n"
            % (start_time - time.time(), time_for_reading, time_for_sampling))
    f.close()


In [19]:
def check_the_pid_(args):
    return check_the_pid(*args)

In [37]:
def several_files():
    files = []
    variant = int(input("Press 1 if you want to use 1 file, press 2 - if different"))
    if variant == 2:
        num = int(input("How many files to use? "))
        for i in range(num):
            files.append(str(input("Enter file name : ")))
    if variant == 1:
        files.append(str(input("Enter file name : ")))
    else:
        print("Invalid input")
        exit()
    return files

In [39]:
%%time
import multiprocessing
from multiprocessing import Pool
from functools import partial
PROCESSES = 8
N = 100
files = ['example'] #, 'example', 'example']#several_files()
parameters = [(i, N) for i in files]
if len(files) > 1: PROCESSES = len(files)
if len(files) == 1: parameters *= PROCESSES
print(parameters)
fp = partial(check_the_pid, N=N)
with multiprocessing.Pool(PROCESSES) as pool:
    pool.map(check_the_pid, parameters)

[('example', 100), ('example', 100), ('example', 100), ('example', 100), ('example', 100), ('example', 100), ('example', 100), ('example', 100)]
CPU times: user 179 ms, sys: 86.4 ms, total: 266 ms
Wall time: 4min 42s


In [40]:
!ls


cpu-time-monoprocess-4vcpu.png	ForkPoolWorker-31-23619.txt
cpu-time-monoprocess-8cpu.png	ForkPoolWorker-32-23620.txt
cpu-time-polyprocess-4vcpu.png	ForkPoolWorker-923270.txt
example				ForkProcess-13.txt
filtering.c			ForkProcess-17.txt
filtering.py			ForkProcess-1.txt
filtering.pyx			ForkProcess-5.txt
flitering.ipynb			ForkProcess-9.txt
fork-ex.c			generate-random.sh
ForkPoolWorker-1723330.txt	latest-commit
ForkPoolWorker-1.txt		MainProcess.txt
ForkPoolWorker-25-23613.txt	mc-approximation.py
ForkPoolWorker-26-23614.txt	monte-carlo-ex.c
ForkPoolWorker-27-23615.txt	parallel-notebook.ipynb
ForkPoolWorker-28-23616.txt	read-hex-to-uint.sh
ForkPoolWorker-29-23617.txt	threads-ex.c
ForkPoolWorker-30-23618.txt


In [10]:
%%time
for i in range(PROCESSES):
    check_the_pid(parameters[i])

[-1092056367, 1006554582, 171063601, 840957311, -710892301, -301284384, -1231963340, 489375044, -490450096, -2135830831, 1950041440, -503125113, -1639974830, 1570204111, -75748685, -528513982, -1857074869, 1894951609, 1019421123, -953366252, 33044647, -158978848, -198407566, -1920484449, -324256890, -1247778941, 262544929, -336654498, -527067087, -1871717194, 826841243, 623294638, 24660898, 186524022, 905105739, 644741141, -1286245736, -1430279130, -1574713543, -36737963, 168238621, 813730202, 271545394, -1201581829, 928709660, -1667011686, -1184394817, -563026442, 1283066431, -2053344603, 2136775895, 2044741581, 533854357, 1768218196, -1762728935, -464993285, 1087793824, 2042920686, -1689013780, -532896657, -1900365890, -633278304, 1975301181, -980544535, -143383379, -32995825, -1405425045, -679461956, -974397494, -600188529, -1121511871, -1363562625, -731260070, 1712473040, 919675731, -938848318, -1938878886, -12801363, -743090956, 1479676921, 1094022365, 216172736, 2125613771, 16088

[1276053424, 1277961659, -633278304, -1247002037, 1189203065, 1494364222, -281392470, 6221037, 1926060046, -679509989, -45600024, 605086302, -569800161, 1727728355, -1038545802, -1635324483, -1039343362, 700471244, 616910817, 309369220, 223666043, 2014716511, -1710068745, -865308569, 939859990, 114845948, 1541401781, 1390197470, -1344038394, 1405219622, 172516056, 947601171, 1742387465, 1906121280, 1132080565, -32170151, 254625764, 1948030938, -47950039, 1232008645, 1791737208, -258631934, 1698550286, 2074657770, 730099619, -1360451358, 1350735003, -1628222453, 760381283, 634440948, 1358677436, 1899858821, -1460942417, -151009405, -1476406404, 254485622, -577173807, -1650211343, 1319380784, -1297842006, -1463716715, 1271580400, 1703219637, 1525760789, -492263787, 1991842898, -549991622, 2137067820, -192095220, 658463139, 86884586, 771727309, -1137978593, 1657279683, 1822241218, 1727909709, -1524925873, 872788850, -1763576120, -1554295330, 1335643647, 1192528957, 1438182263, -148965605,

In [42]:
import _thread
import threading


In [44]:


class myThread(threading.Thread):
    def __init__(self, threadID, parameters):#, lock):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = "Thread-" + str(threadID)
        self.filename, self.number = parameters
        self.lock = lock
    def run(self):
        print ("Starting " + self.name)
        thread_task(self.name, self.filename, self.number)#, self.lock)
        print ("Exiting " + self.name)

def thread_task(name, filename, N):#, lock):
    #lock.acquire() 
    filtering(name, filename, N)
    #lock.release()



In [45]:
def filtering(name, filename, N):
    start_time = time.time()
    x  = read_4bytes(filename)
    time_for_reading = time.time() - start_time
    subsample = make_subsample(N, x)
    time_for_sampling = time.time() - time_for_reading
    thread_file = (str(name)+".txt")
    f = open(thread_file, "a+")
    pid = int(os.getpid())
    f.write("Process ID : %d\nParent ID : %d\n" % (os.getppid(), pid))
    res = []
    for i in subsample:
        f.write("%d\n" % (i))
        if i % (3 * pid) == 0:
            res.append(i)
    if res:
        for i in res: f.write("%d\n" % (i))
    f.write("Execution time: %s\nTime spent for reading file: %s\nTime spent fot sampling: %s\n"
            % (str(time.time() - start_time ), str(time_for_reading), str(time_for_sampling)))
    f.close()

In [48]:
%%time
#def creating_threads():
THREADS = 8
N = 100
files = ['example'] #several_files()
parameters = [(i, N) for i in files]
if len(files) > 1: THREADS = len(files)
if len(files) == 1: parameters *= THREADS

# Create new threads
threadID = 1
name = "Thread-"
#lock = 0 # threading.Lock() 
threads = []
for parameter in parameters:
    thread = myThread(threadID, parameter)#, lock)
    thread.start()
    threads.append(thread)
    threadID += 1

for thread in threads:
    thread.join()

print ("Exiting Main Thread")

Starting Thread-1
Starting Thread-2
Starting Thread-3
Starting Thread-4
Starting Thread-5
Starting Thread-6
Starting Thread-7
Starting Thread-8
Exiting Thread-2
Exiting Thread-4
Exiting Thread-5
Exiting Thread-3
Exiting Thread-6
Exiting Thread-1
Exiting Thread-8
Exiting Thread-7
Exiting Main Thread
CPU times: user 13min 14s, sys: 5.33 s, total: 13min 19s
Wall time: 13min 10s
